In [3]:
#import
from pyspark.sql import SparkSession
import os
import glob

# Initialize SparkSession
spark = SparkSession.builder.appName("TPC-DS Data Loading").config("spark.sql.catalogImplementation", "hive").getOrCreate()

In [4]:
spark.catalog.listTables()

[Table(name='call_center', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='catalog_page', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='catalog_returns', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='catalog_sales', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='customer', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='customer_address', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='customer_demographics', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='date_dim', catalog='spark_cata

In [5]:
#create all tables
with open(os.getcwd() + "/Create_database.sql", 'r') as file: # path/to/tpcds.sql
    db_string = file.read()

db_string = str.split(db_string, ";")
db_string = db_string[:-1]

for string in db_string:
    if string != '\n' or string != '':
        spark.sql(string)

In [13]:
#load datas into tables
data_file = glob.glob(os.getcwd() + "/data/*")
for i in data_file:
    table_name = i.split("/")[-1].split(".")[0]
    if table_name != "dbgen_version":
        sql_command = "select * from " + table_name
        tmp=spark.sql(sql_command)
        #print(tmp)
        df = spark.read.schema(tmp.schema).csv( i, sep='|')
        df.write.mode("append").insertInto(table_name)
        print("Data from table" + table_name + "is inserted")

In [1]:
#run queries
queries_files = glob.glob(os.getcwd() + "/queries/queries*")
for i in queries_files:
    with open( i , 'r') as file:
    db_string = file.read()
    
    #start time
    try:
        spark.sql(db_string)
    except:
        print( i + "doesn't work")
    #end time
    


In [ ]:
spark.sql("select count(*) from customer").show()
